# 02 - Unity Catalog Connection Setup

This notebook configures Unity Catalog connections for SFTP data sources and verifies AutoLoader can read from SFTP.

**Note:** AutoLoader with SFTP is straightforward and built-in to Databricks. The main purpose of this repository is to demonstrate the **custom SFTPWriter** for writing data back to SFTP servers.

This notebook will:
- Create Unity Catalog SFTP connections (stores credentials securely)
- Verify AutoLoader can read from SFTP using simple URI format
- Create catalog and schema structure for the DLT pipeline

## 1. Load Configuration from Previous Setup

In [0]:
# Create widgets for catalog and schema configuration
dbutils.widgets.text("catalog_name", "sftp_demo", "Catalog Name")
dbutils.widgets.text("schema_name", "default", "Schema Name")
dbutils.widgets.text("source_connection_name", "source_sftp_connection", "Source Connection Name")
dbutils.widgets.text("target_connection_name", "target_sftp_connection", "Target Connection Name")

# Get widget values
CATALOG_NAME = dbutils.widgets.get("catalog_name")
SCHEMA_NAME = dbutils.widgets.get("schema_name")
SOURCE_CONNECTION_NAME = dbutils.widgets.get("source_connection_name")
TARGET_CONNECTION_NAME = dbutils.widgets.get("target_connection_name")

print(f"Catalog: {CATALOG_NAME}")
print(f"Schema: {SCHEMA_NAME}")
print(f"Source Connection: {SOURCE_CONNECTION_NAME}")
print(f"Target Connection: {TARGET_CONNECTION_NAME}")

In [ ]:
# Load configuration
config_df = spark.table(f"{CATALOG_NAME}.config.connection_params")
config_dict = {row.key: row.value for row in config_df.collect()}

# Get configuration values
catalog_name = config_dict.get("catalog_name", CATALOG_NAME)
schema_name = config_dict.get("schema_name", SCHEMA_NAME)
source_connection_name = config_dict.get("source_connection_name", SOURCE_CONNECTION_NAME)
target_connection_name = config_dict.get("target_connection_name", TARGET_CONNECTION_NAME)

source_host = config_dict["source_host"]
source_username = config_dict["source_username"]
target_host = config_dict["target_host"]
target_username = config_dict["target_username"]
secret_scope = config_dict["secret_scope"]
ssh_key_secret = config_dict["ssh_key_secret"]

print("Configuration loaded successfully")
print(f"Catalog: {catalog_name}")
print(f"Schema: {schema_name}")
print(f"Source Connection: {source_connection_name}")
print(f"Target Connection: {target_connection_name}")
print(f"Secret scope: {secret_scope}")
print(f"SSH key secret: {ssh_key_secret}")

## 2. Create Unity Catalog Connection for Source SFTP

**Note:** This requires Databricks workspace admin privileges.

In [ ]:
# Set catalog context first
spark.sql(f"USE CATALOG {CATALOG_NAME}")

# Create source SFTP connection
spark.sql(f"""
CREATE CONNECTION IF NOT EXISTS {SOURCE_CONNECTION_NAME}
TYPE sftp
OPTIONS (
  host '{source_host}',
  port '22',
  user SECRET ('{secret_scope}', 'source-username'),
  pem_private_key SECRET ('{secret_scope}', '{ssh_key_secret}')
)
""")

print(f"Source SFTP connection created: {SOURCE_CONNECTION_NAME} (in catalog {CATALOG_NAME})")

## 3. Create Unity Catalog Connection for Target SFTP

In [ ]:
# Create target SFTP connection
spark.sql(f"""
CREATE CONNECTION IF NOT EXISTS {TARGET_CONNECTION_NAME}
TYPE sftp
OPTIONS (
  host '{target_host}',
  port '22',
  user SECRET ('{secret_scope}', 'target-username'),
  pem_private_key SECRET ('{secret_scope}', '{ssh_key_secret}')
)
""")

print(f"Target SFTP connection created: {TARGET_CONNECTION_NAME} (in catalog {CATALOG_NAME})")

## 4. Verify AutoLoader with SFTP

Test that AutoLoader can read from SFTP using the Unity Catalog connection.

In [ ]:
# Test reading customers.csv from source SFTP using AutoLoader
source_sftp_uri = f"sftp://{source_username}@{source_host}:22/customers.csv"

customers_df = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "csv")
    .option("cloudFiles.schemaLocation", f"/tmp/{CATALOG_NAME}/schema/customers")
    .option("header", "true")
    .load(source_sftp_uri)
)

# Display schema
customers_df.printSchema()

# Write to temporary table for verification
(
    customers_df.writeStream
    .format("memory")
    .queryName("test_customers")
    .outputMode("append")
    .start()
)

print(f"Source SFTP AutoLoader verified successfully using URI: {source_sftp_uri}")

## 5. Create Catalog Structure for DLT Pipeline

Create the catalog and schema structure for the Delta Live Tables pipeline.

In [0]:
# Create catalog structure for DLT pipeline
spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG_NAME}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.bronze")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.silver")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.gold")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG_NAME}.{SCHEMA_NAME}")

print("Catalog structure created:")
print(f"  - {CATALOG_NAME}.bronze (raw data from source SFTP)")
print(f"  - {CATALOG_NAME}.silver (cleaned and validated data)")
print(f"  - {CATALOG_NAME}.gold (aggregated business-level data)")
print(f"  - {CATALOG_NAME}.{SCHEMA_NAME} (default schema)")